# COBS Decoder Simulation Output Comparator

This program is a companion to COBS_dec-tv-generator, which creates a list of randomly-generated frames and then generates test vectors for the COBS Decoder simulation.

We read in the original randomly-generated frames data structure from the file where COBS_dec-tv-generator saved it. This file is a simple serialization (using `pickle`) of the data structure used in both programs for the variable `unencoded_frames`.

We read in the simulation test results from the file where the Vivado simulation saved them. This is a text file with one row per clock cycle. Rows beginning with `--` are comments and are ignored. All other rows are expected to contain binary representations of the input and output signals of the decoder entity, where each binary digit can be 0, 1, or U for undetermined.

This program simulates the simple behavior of the AXI-Stream consumer connected to the COBS decoder's output. In other words, it accepts an input byte from `m_tdata` (signal names given from the decoder's perspective) when and only when both of `m_tvalid` and `m_tready` are 1. That byte is taken to be the last byte of a frame if `m_tlast` is also 1. In this way, we easily build up the frames decoded by the simulated decoder entity.

The two data structures are then compared, and a simple report generated.

In [ ]:
import pickle
import re
import sys

In [ ]:
# Filenames. These should be command-line arguments.

input_filename = 'COBS_dec-tv-frames-67c041aa'  # 1000-frames test that worked the first time
simulation_filename = 'cobs-test-output.txt'

In [ ]:
# Read in the input file to recreate the random frames in the test vectors

with open(input_filename, "rb") as f:
    input_frames = pickle.load(f)

## print(f'{len(input_frames)} original frames read')

In [ ]:
# Parse the simulation results file and extract frames

# Fields in the simulation results file:
#     0      1        2        3        4          5         6       7        8
# -- rst input_data s_tlast s_tvalid s_tready output_data m_tlast m_tvalid m_tready

sim_frames = []
frame = bytearray()
frame_line_numbers = []
sim_frames_line_numbers = []
line_no = 0

with open(simulation_filename, "r") as f:
    for line in f:
        line_no += 1
        fields = re.findall("[01U]+", line)
        if len(fields) != 9:
            if not re.match("--", line):
                print(f"Error: invalid line {line_no}: {line}")
            continue
        if len(fields[5]) != 8 or len(fields[6]) != 1 or len(fields[7]) != 1 or len(fields[8]) != 1:
            print(f"Error: invalid field length on line {line_no}: {line}")
        m_tdata_binary = fields[5]
        m_tlast = True if fields[6][0] == '1' else False
        m_tvalid = True if fields[7][0] == '1' else False
        m_tready = True if fields[8][0] == '1' else False

        if m_tready and m_tvalid:
            frame += bytes([int(m_tdata_binary, 2)])
            frame_line_numbers.append(line_no)
            if m_tlast:
                sim_frames += [frame]
                sim_frames_line_numbers += [frame_line_numbers]
                # print(f"Ending {len(frame)} byte frame at line {line_no}")
                frame = bytearray()
                frame_line_numbers = []
        

In [ ]:
# Compare simulation results to original random frames

match = True

if len(sim_frames) != len(input_frames):
    print(f"Number of frames does not match. Original: {len(input_frames)} Sim: {len(sim_frames)}")
    match = False

for frame, sim_frame, frame_line_numbers in zip(input_frames, sim_frames, sim_frames_line_numbers):
    if len(frame) != len(sim_frame):
        print(f"Length of frame starting at line {frame_line_numbers[0]} is {len(sim_frame)}; should be {len(frame)}.")
        match = False
    for fbyte, sbyte, lineno in zip(frame, sim_frame, frame_line_numbers):
        if fbyte != sbyte:
            print(f"Byte value at line {lineno} is hex {sbyte:02x}; should be hex {fbyte:02x}")
            match = False

if match:
    print(f'Simulation results match original data.')
    print(f'{len(input_frames)} frames compared.')

In [ ]:
# Return success or failure to the shell
# (This will throw an exception when run in the Jupyter notebook environment.)

if match:
    sys.exit(0)
else:
    sys.exit(1)